In [19]:
import pandas as pd
import numpy as np
import os
import joblib
import logging
from datetime import datetime
import boto3
from decimal import Decimal
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)

# ---------------- Configuration ----------------
TRAINING_FILE = "Final_dataset_1000.csv"  # Replace with your S3 local download if needed
MODEL_PATH = "models/multi_task_model.joblib"
DYNAMODB_TABLE = "Final_predictions"
USE_DYNAMODB = False  # Set True if you want to upload

# Initialize DynamoDB client if needed
if USE_DYNAMODB:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table(DYNAMODB_TABLE)

# ---------------- Load Data ----------------
logging.info("📂 Loading training/prediction data...")
df = pd.read_csv(TRAINING_FILE)
logging.info(f"✅ Loaded {len(df)} rows.")

# ---------------- Define Targets and Features ----------------
target_cols = [
    'overall_risk_score_pct','heart_risk_score_pct','hypertension_risk_score_pct',
    'diabetes_risk_score_pct','respiratory_risk_score_pct'
]

# Keep numeric columns only for features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in target_cols]

logging.info(f"🔹 Feature columns: {feature_cols}")
logging.info(f"🔹 Target columns: {target_cols}")

# ---------------- Train Model ----------------
logging.info("🚀 Training multi-output regression model...")
X = df[feature_cols]
y = df[target_cols]

# Simple train-test split (optional)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
logging.info("✅ Model trained.")

# Save model
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
joblib.dump({
    "model": model,
    "feature_cols": feature_cols,
    "target_cols": target_cols
}, MODEL_PATH)
logging.info(f"💾 Model saved to {MODEL_PATH}")

# ---------------- Predict ----------------
logging.info("🛰 Predicting on input data...")
preds = model.predict(X)

df_preds = pd.DataFrame(preds, columns=target_cols)

# ---------------- Generate Recommendations ----------------
def generate_recommendations(row):
    recs = {}

    # Exercise
    if row['overall_risk_score_pct'] < 50:
        recs['recommendation_exercise'] = "Exercise daily 30 mins, light intensity"
    else:
        recs['recommendation_exercise'] = "Exercise daily 60 mins, moderate to high intensity"

    # Diet
    if row['Diet_risk_index'] <= 1:
        recs['recommendation_diet'] = "Maintain balanced diet with high protein and fiber"
    else:
        recs['recommendation_diet'] = "Reduce sugar/processed foods, increase vegetables & proteins"

    # Sleep
    if row['sleep_hours_per_day'] >= 6 and row['sleep_hours_per_day'] <= 8:
        recs['recommendation_sleep'] = f"Maintain regular sleep schedule of {row['sleep_hours_per_day']} hours"
    else:
        recs['recommendation_sleep'] = "Aim for 7-8 hours of sleep daily"

    # Lifestyle
    if row['stress_level'] <= 4:
        recs['recommendation_lifestyle'] = "Continue current lifestyle and stay active"
    else:
        recs['recommendation_lifestyle'] = "Incorporate stress management practices like meditation or yoga"

    return pd.Series(recs)

df_recs = df.apply(generate_recommendations, axis=1)

# ---------------- Combine ----------------
df_final = pd.concat([df.reset_index(drop=True), df_preds, df_recs], axis=1)

# ---------------- Upload to DynamoDB ----------------
if USE_DYNAMODB:
    for _, row in df_final.iterrows():
        item = row.to_dict()
        # Convert floats to Decimal for DynamoDB
        for k, v in item.items():
            if isinstance(v, float):
                item[k] = Decimal(str(round(v, 4)))
        table.put_item(Item=item)
    logging.info(f"✅ Uploaded {len(df_final)} records to DynamoDB.")

# ---------------- Save locally ----------------
df_final.to_csv("Final_predictions_with_recommendations.csv", index=False)
logging.info("💾 Saved predictions + recommendations to Final_predictions_with_recommendations.csv")
logging.info("🏁 Script completed successfully.")


INFO:root:📂 Loading training/prediction data...
INFO:root:✅ Loaded 1000 rows.
INFO:root:🔹 Feature columns: ['age', 'height_cm', 'weight_kg', 'physical_activity_level', 'sleep_hours_per_day', 'stress_level', 'cholesterol_level', 'glucose_level', 'blood_pressure_sys', 'blood_pressure_dia', 'BMI_derived', 'Hypertension_flag', 'Prediabetes_Diabetes_flag', 'Physical_inactivity_flag', 'Cardiac_risk_index', 'Stress_index', 'Diet_risk_index', 'Lung_risk_index']
INFO:root:🔹 Target columns: ['overall_risk_score_pct', 'heart_risk_score_pct', 'hypertension_risk_score_pct', 'diabetes_risk_score_pct', 'respiratory_risk_score_pct']
INFO:root:🚀 Training multi-output regression model...
INFO:root:✅ Model trained.
INFO:root:💾 Model saved to models/multi_task_model.joblib
INFO:root:🛰 Predicting on input data...
INFO:root:💾 Saved predictions + recommendations to Final_predictions_with_recommendations.csv
INFO:root:🏁 Script completed successfully.


In [20]:
import pandas as pd
import numpy as np
import os
import joblib
import logging
import boto3
from decimal import Decimal
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)

# ---------------- Configuration ----------------
S3_BUCKET = "wellifyyyy"
S3_INPUT_PREFIX = "processed_for_sagemaker/"
LOCAL_MODEL_DIR = "models"
MODEL_FILENAME = "multi_task_model.joblib"
MODEL_PATH = os.path.join(LOCAL_MODEL_DIR, MODEL_FILENAME)
DYNAMODB_TABLE = "Final_predictions"
USE_DYNAMODB = True  # Set False if you don't want to upload

# Initialize clients
s3 = boto3.client("s3")
if USE_DYNAMODB:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table(DYNAMODB_TABLE)

# ---------------- Helper Functions ----------------
def load_latest_s3_file(bucket, prefix):
    """Get the latest CSV file from S3 prefix."""
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    files = [obj['Key'] for obj in resp.get('Contents', []) if obj['Key'].endswith('.csv')]
    if not files:
        raise FileNotFoundError("No CSV files found in S3 prefix")
    latest_file = sorted(files)[-1]
    logging.info(f"📂 Latest input file: {latest_file}")
    obj = s3.get_object(Bucket=bucket, Key=latest_file)
    return pd.read_csv(obj['Body'])

def generate_recommendations(row):
    """Generate lifestyle recommendations based on predicted risk scores."""
    recs = {}

    # Exercise recommendation
    if row.get('overall_risk_score_pct', 0) < 50:
        recs['recommendation_exercise'] = "Exercise daily 30 mins, light intensity"
    else:
        recs['recommendation_exercise'] = "Exercise daily 60 mins, moderate to high intensity"

    # Diet recommendation
    if row.get('Diet_risk_index', 0) <= 1:
        recs['recommendation_diet'] = "Maintain balanced diet with high protein and fiber"
    else:
        recs['recommendation_diet'] = "Reduce sugar/processed foods, increase vegetables & proteins"

    # Sleep recommendation
    sleep_hours = row.get('sleep_hours_per_day', 7)
    if 6 <= sleep_hours <= 8:
        recs['recommendation_sleep'] = f"Maintain regular sleep schedule of {sleep_hours} hours"
    else:
        recs['recommendation_sleep'] = "Aim for 7-8 hours of sleep daily"

    # Lifestyle / stress recommendation
    stress = row.get('stress_level', 3)
    if stress <= 4:
        recs['recommendation_lifestyle'] = "Continue current lifestyle and stay active"
    else:
        recs['recommendation_lifestyle'] = "Incorporate stress management practices like meditation or yoga"

    return pd.Series(recs)

# ---------------- Main ----------------
def main():
    logging.info("🚀 Starting workflow...")

    # Load input data
    df_input = load_latest_s3_file(S3_BUCKET, S3_INPUT_PREFIX)
    logging.info(f"✅ Loaded {len(df_input)} records")

    # Select numeric columns
    target_cols = [
        'overall_risk_score_pct','heart_risk_score_pct','hypertension_risk_score_pct',
        'diabetes_risk_score_pct','respiratory_risk_score_pct'
    ]
    numeric_cols = df_input.select_dtypes(include=[np.number]).columns.tolist()
    feature_cols = [c for c in numeric_cols if c not in target_cols]
    logging.info(f"🔹 Feature columns: {feature_cols}")
    logging.info(f"🔹 Target columns: {target_cols}")

    # ---------------- Train or load model ----------------
    if os.path.exists(MODEL_PATH):
        logging.info(f"📂 Loading existing model from {MODEL_PATH}")
        model_artifacts = joblib.load(MODEL_PATH)
        model = model_artifacts['model']
        feature_cols = model_artifacts['feature_cols']
        target_cols = model_artifacts['target_cols']
    else:
        logging.info("🚀 Training multi-output regression model...")
        X = df_input[feature_cols]
        y = df_input[target_cols]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        os.makedirs(LOCAL_MODEL_DIR, exist_ok=True)
        joblib.dump({
            "model": model,
            "feature_cols": feature_cols,
            "target_cols": target_cols
        }, MODEL_PATH)
        logging.info(f"✅ Model trained and saved to {MODEL_PATH}")

    # ---------------- Predict ----------------
    logging.info("🛰 Predicting risk scores...")
    X_input = df_input[feature_cols]
    preds = model.predict(X_input)
    df_preds = pd.DataFrame(preds, columns=target_cols)

    # ---------------- Generate recommendations ----------------
    df_with_preds = pd.concat([df_input.reset_index(drop=True), df_preds], axis=1)
    df_recs = df_with_preds.apply(generate_recommendations, axis=1)

    # ---------------- Combine ----------------
    df_final = pd.concat([df_with_preds, df_recs], axis=1)

    # ---------------- Upload to DynamoDB ----------------
    if USE_DYNAMODB:
        for _, row in df_final.iterrows():
            item = row.to_dict()
            for k, v in item.items():
                if isinstance(v, float):
                    item[k] = Decimal(str(round(v, 4)))
            table.put_item(Item=item)
        logging.info(f"✅ Uploaded {len(df_final)} records to DynamoDB table {DYNAMODB_TABLE}")

    # ---------------- Save locally ----------------
    df_final.to_csv("Final_predictions_with_recommendations.csv", index=False)
    logging.info("💾 Saved predictions + recommendations locally")
    logging.info("🏁 Workflow completed successfully.")

if __name__ == "__main__":
    main()


INFO:root:🚀 Starting workflow...
INFO:root:📂 Latest input file: processed_for_sagemaker/part-00000-657ae969-76b2-4c0e-b9f0-17850b887606-c000.csv
INFO:root:✅ Loaded 28 records
INFO:root:🔹 Feature columns: ['age', 'height_cm', 'weight_kg', 'physical_activity_level', 'sleep_hours_per_day', 'stress_level', 'cholesterol_level', 'glucose_level', 'blood_pressure_sys', 'blood_pressure_dia', 'BMI_derived', 'Hypertension_flag', 'Prediabetes_Diabetes_flag', 'Physical_inactivity_flag', 'Cardiac_risk_index', 'Stress_index', 'Diet_risk_index', 'Lung_risk_index']
INFO:root:🔹 Target columns: ['overall_risk_score_pct', 'heart_risk_score_pct', 'hypertension_risk_score_pct', 'diabetes_risk_score_pct', 'respiratory_risk_score_pct']
INFO:root:📂 Loading existing model from models/multi_task_model.joblib
INFO:root:🛰 Predicting risk scores...
INFO:root:✅ Uploaded 28 records to DynamoDB table Final_predictions
INFO:root:💾 Saved predictions + recommendations locally
INFO:root:🏁 Workflow completed successfully.
